In [12]:
# Import libraries
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction
using Missings

os = pyimport("os")
pyimport("scienceplots")
np = pyimport("numpy")


PyCall.pygui(:tk)

FONTSIZE=20.5;

In [2]:
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
###############################                Parameter Learning Tables             ###########################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################

In [16]:
function get_up_low(post_dict, upper_quant, lower_quant, γ0_is_log)
    p = [upper_quant,lower_quant]
    quantiles = zeros(19, 6, 2)
    sorted_keys = sort(collect(keys(post_dict)))
    # Adjust for when gamma_0 is considered on the log scale

        for i in 1:19
            mat = post_dict[ sorted_keys[i] ]
            quantiles[i, 1,:] = quantile( mat[:,1], p )
            quantiles[i, 2,:] = quantile( mat[:,2], p )
            quantiles[i, 3,:] = quantile( mat[:,3], p )
            quantiles[i, 4,:] = quantile( mat[:,4], p )
            quantiles[i, 5,:] = (γ0_is_log) ? quantile( np.log(mat[:,5]), p ) : quantile( mat[:,5], p )
            quantiles[i, 6,:] = quantile( mat[:,6], p )
        end

    return quantiles
end

Realizations = [string(i) for i in 1:100];
cal_years = collect(range(2015,step=15,length=20));

# Load the present day posteriors 
gamma_0_is_log = true
present_posterior = np.load("../Data/Training_Data/posterior_samples_All_Combined.npy");

# Compute upper and lower quantile bounds for present day posteriors
present_bounds = []
present_widths = []
for i in 1:6
    lower = (i == 5 && gamma_0_is_log) ? quantile(np.log(present_posterior[:,i]), 0.05) : quantile(present_posterior[:,i], 0.05)
    upper = (i == 5 && gamma_0_is_log) ? quantile(np.log(present_posterior[:,i]), 0.95) : quantile(present_posterior[:,i], 0.95)
    push!(present_bounds, (lower,upper))
    push!(present_widths, upper - lower)
end
present_bounds = reshape(present_bounds, 1, :)
present_widths = reshape(present_widths, 1, :);


# Change to wherever your posterior dictionaries are located
path_to_posteriors = "../Data/Posterior_Data"

# Initialize empty vectors to hold interval bounds and widths 
all_widths = zeros(length(Realizations),19,6)
all_bounds = zeros(length(Realizations), 19, 6, 2)


# Loop to calculate quantiles and credible interval widths
    for (iter, r) in enumerate(Realizations)
        post = JLD2.load("$(path_to_posteriors)/R_$(r)_Posterior_Dict.jld2", "post_data")
        cred_int_90_up_low = get_up_low(post, 0.95, 0.05,gamma_0_is_log)
        all_bounds[iter, :, :, :] = cred_int_90_up_low
        all_widths[iter,:,:] = cred_int_90_up_low[:,:,1] .- cred_int_90_up_low[:,:,2]
    end
# The average 5th percentile and and 95th percentiles of the six MALI parameters across all trajectories over the 19 years of calibration
avg_bounds = mean(all_bounds,dims=1)
avg_bounds = dropdims(avg_bounds, dims=1)

# The average 90% credible interval widths of the six MALI parameters across all trajectories for each of the 19 years of calibration
avg_widths = mean(all_widths,dims=1)
avg_widths = dropdims(avg_widths, dims=1)
println(size(avg_widths))
# The 17% and 83% quantiles of the 90% cred int widths of the six MALI parameters across all trajectories for each of the 19 years of calibration
width_variation_interval = np.quantile(all_widths, 0.83,axis=0) .- np.quantile(all_widths, 0.17,axis=0)


# Store the bounds as tuples in one matrix
bounds_as_tuples = fill((0.0,0.0),(19,6))
for i in 1:19
    for j in 1:6
        bounds_as_tuples[i,j] = (avg_bounds[i,j,2], avg_bounds[i,j,1])
    end
end
# Add the 2015 90% credible interval bounds
bounds_w_present = vcat(present_bounds, bounds_as_tuples);

# Add the 2015 90% credible widths
widths_w_present = vcat(present_widths, avg_widths);


# Add the years of calibration as another column
bounds_w_years = hcat(cal_years, bounds_w_present)
widths_w_years = hcat(cal_years, widths_w_present)
width_var_w_years = hcat(cal_years[2:end], width_variation_interval)

# Dataframe column names
names = ["Latest_calibration_year","vmThresh","fricExp","mu_scale","stiff_scale","gamma0","melt_flux"]

# Covnert to datframe before saving to disc
params_df_bounds = DataFrame(bounds_w_years, names)
params_df_widths = DataFrame(widths_w_years, names)
params_df_width_var = DataFrame(width_var_w_years, names)

#Calculate total percent decrease of 90% cred interval width
pct_change = ((collect(params_df_widths[1,2:end]) .- collect(params_df_widths[20,2:end])) ./ collect(params_df_widths[1,2:end])) .* 100
# Make a 1×7 row: first element is label, then pct_change values
new_row = hcat("total_%_decrease", pct_change...)
# Convert to DataFrame with the same column names
new_df = DataFrame(new_row, names)
# Append to params_df_widths
append!(params_df_widths, new_df)


CSV.write("../Data/Parameter_Learning_Tables/CI_bounds_log_gamma0_$(gamma_0_is_log).csv",params_df_bounds)
CSV.write("../Data/Parameter_Learning_Tables/CI_widths_log_gamma0_$(gamma_0_is_log).csv",params_df_widths)
CSV.write("../Data/Parameter_Learning_Tables/CI_width_variation_log_gamma0_$(gamma_0_is_log).csv",   params_df_width_var)

(19, 6)


Row,Latest_calibration_year,vmThresh,fricExp,mu_scale,stiff_scale,gamma0,melt_flux
,Any,Any,Any,Any,Any,Any,Any
1,2015,34089.3,0.172232,0.232868,0.205653,2.53179,30.4802
2,2030,33780.8,0.167235,0.21296,0.195873,2.46702,30.3705
3,2045,33765.3,0.15895,0.208421,0.190561,2.25623,30.1791
4,2060,33642.9,0.157167,0.206643,0.186847,1.8003,29.8141
5,2075,33475.8,0.153437,0.203493,0.181547,1.24637,28.1198
6,2090,33153.9,0.151732,0.198547,0.175937,1.2052,26.9038
7,2105,32615.3,0.145927,0.181701,0.164678,1.13852,26.5008
8,2120,32367.7,0.139435,0.175612,0.161105,1.02016,26.2051
9,2135,31971.1,0.135851,0.172647,0.154908,0.898344,24.2433


In [21]:
##CALCULATING THE YEAR BY YEAR CHANGE IN UNCERTAINTY RANGES FOR EACH PARAMETER, AS WELL AS PERCENT CONTRIBUTION TO TOTAL LEARNING
##
cal_years = collect(range(2015,step=15,length=20))

Δ_param_width = zeros(20, 6)
Δ_param_width_pct = zeros(20, 6)

#Total learning from 2015 - 2300

# find row indices
total_idx1 = only(findall(x -> x == 2015, cal_years))
total_idx2 = only(findall(x -> x == 2300, cal_years))

# extract vectors (dropping year column)
present = collect(params_df_widths[total_idx1, :])[2:end]
final = collect(params_df_widths[total_idx2, :])[2:end]

# compute differences
total_diff = final .- present
Δ_param_width[20, :] = total_diff
#Δ_param_width_pct[20,:] .= 100.0

learn_periods = []

for i in 1:(length(cal_years)-1)
    yr1 = cal_years[i]
    yr2 = cal_years[i+1]

    # find row indices
    row_idx1 = only(findall(x -> x == yr1, cal_years))
    row_idx2 = only(findall(x -> x == yr2, cal_years))

    # extract vectors (dropping year column)
    row1 = collect(params_df_widths[row_idx1, :])[2:end]
    row2 = collect(params_df_widths[row_idx2, :])[2:end]

    # compute differences
    diff = row2 .- row1
    Δ_param_width[i, :] = diff
    Δ_param_width_pct[i, :] = (diff ./ total_diff) .* 100
    # Collect this learning period as string for datframe labels
    push!(learn_periods, "$(yr1) - $(yr2)")
end

# Add the total learning period
push!(learn_periods, "2015 - 2300")
width_change_mat = zeros(20,12)

# Put the parameter width changes and percent changes together param by param columnwise
for i in 1:6
    width_change_mat[:,(2*i)-1] = Δ_param_width[:,i]
    width_change_mat[:,(2*i)] = Δ_param_width_pct[:,i]
end


names = ["LearningPeriod","ΔvmThresh","Δ_%_vmThresh","Δ_fricExp","Δ_%_fricExp","Δ_mu_scale","Δ_%_mu_scale","Δ_stiff_scale","Δ_%_stiff_scale","Δ_gamma0","Δ_%_gamma0","Δ_melt_flux","Δ_%_melt_flux"]
width_change_w_years = hcat(learn_periods, width_change_mat)
df_widths_yr_to_yr = DataFrame(width_change_w_years, names)
allowmissing!(df_widths_yr_to_yr)
df_widths_yr_to_yr[20, [3,5,7,9,11,13]] .= missing
CSV.write("../Data/Parameter_Learning_Tables/yr_to_yr_change_in_widths_log_gamma0_$(gamma_0_is_log).csv", df_widths_yr_to_yr)

"../Data/Parameter_Learning_Tables/yr_to_yr_change_in_widths_log_gamma0_true.csv"

In [5]:
println(sum(df_bounds_yr_to_yr[1:5, 11]) + sum(df_bounds_yr_to_yr[6:13, 11]) + sum(df_bounds_yr_to_yr[14:19, 11]))

100.00000000000001


In [6]:
cols = [3,5,7,9,11,13]
for (i,col) in enumerate(cols)
    println("$(names[2i+1]) decrease from 2015 to 2090: ")
    println(sum(df_bounds_yr_to_yr[1:5, col]))
end

Δ_%_vmThresh decrease from 2015 to 2090: 
11.193044442419605
Δ_%_fricExp decrease from 2015 to 2090: 
13.56602081348913
Δ_%_mu_scale decrease from 2015 to 2090: 
24.11285593408966
Δ_%_stiff_scale decrease from 2015 to 2090: 
25.69691638711342
Δ_%_gamma0 decrease from 2015 to 2090: 
58.146813524449946
Δ_%_melt_flux decrease from 2015 to 2090: 
30.490298144376688


In [7]:
cols = [3,5,7,9,11,13]
for (i,col) in enumerate(cols)
    println("$(names[2i+1]) decrease from 2090 to 2210: ")
    println(sum(df_bounds_yr_to_yr[6:13, col]))
end



Δ_%_vmThresh decrease from 2090 to 2210: 
65.35858638372444
Δ_%_fricExp decrease from 2090 to 2210: 
56.43968759374293
Δ_%_mu_scale decrease from 2090 to 2210: 
65.15053906260987
Δ_%_stiff_scale decrease from 2090 to 2210: 
61.183643628975034
Δ_%_gamma0 decrease from 2090 to 2210: 
36.681516511449466
Δ_%_melt_flux decrease from 2090 to 2210: 
56.939614177713494


In [8]:
cols = [3,5,7,9,11,13]
for (i,col) in enumerate(cols)
    println("$(names[2i+1]) decrease from 2210 to 2300: ")
    println(sum(df_bounds_yr_to_yr[14:19, col]))
end


Δ_%_vmThresh decrease from 2210 to 2300: 
23.448369173855948
Δ_%_fricExp decrease from 2210 to 2300: 
29.99429159276793
Δ_%_mu_scale decrease from 2210 to 2300: 
10.736605003300483
Δ_%_stiff_scale decrease from 2210 to 2300: 
13.119439983911551
Δ_%_gamma0 decrease from 2210 to 2300: 
5.171669964100604
Δ_%_melt_flux decrease from 2210 to 2300: 
12.570087677909815


In [24]:
##CALCULATING THE YEAR BY YEAR CHANGE IN UNCERTAINTY RANGE STANDARD DEVIATION FOR EACH PARAMETER, AS WELL AS PERCENT CONTRIBUTION TO TOTAL CHANGE
##
cal_years = collect(range(2030,step=15,length=19))

Δ_width_var = zeros(18, 6)
Δ_width_var_pct = zeros(18, 6)

learn_periods = []

for i in 1:(length(cal_years)-1)
    yr1 = cal_years[i]
    yr2 = cal_years[i+1]

    # find row indices
    row_idx1 = only(findall(x -> x == yr1, cal_years))
    row_idx2 = only(findall(x -> x == yr2, cal_years))

    # extract vectors (dropping year column)
    row1 = collect(params_df_width_var[row_idx1, :])[2:end]
    row2 = collect(params_df_width_var[row_idx2, :])[2:end]

    # compute differences
    diff = row2 .- row1
    Δ_width_var[i, :] = round.(diff, sigdigits = 6)
    Δ_width_var_pct[i, :] = (diff ./ total_diff) .* 100
    # Collect this learning period as string for datframe labels
    push!(learn_periods, "$(yr1) - $(yr2)")
end
# Add the total learning period (2015 not included since multiple trajectories only began with 2030 caliobration onward)
width_variance_change = zeros(18,12)

# Put the parameter width changes and percent changes together param by param columnwise
for i in 1:6
    width_variance_change[:,(2*i)-1] = Δ_width_var[:,i]
    width_variance_change[:,(2*i)] = Δ_width_var_pct[:,i]
end


names = ["LearningPeriod","ΔvmThresh","Δ_%_vmThresh","Δ_fricExp","Δ_%_fricExp","Δ_mu_scale","Δ_%_mu_scale","Δ_stiff_scale","Δ_%_stiff_scale","Δ_gamma0","Δ_%_gamma0","Δ_melt_flux","Δ_%_melt_flux"]
width_variance_w_years = hcat(learn_periods, width_variance_change)
df_width_variance_yr_to_yr = DataFrame(width_variance_w_years , names)
CSV.write("../Data/Parameter_Learning_Tables/yr_to_yr_change_in_width_variation_log_gamma0_$(gamma_0_is_log).csv", df_width_variance_yr_to_yr)

"../Data/Parameter_Learning_Tables/yr_to_yr_change_in_width_variation_log_gamma0_true.csv"